In [44]:
!pip install vaderSentiment
!pip install pyspark


[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


In [6]:
import plotly.express as px
data = dict(
    character=["travel", "travel", "movie", "sports", "game", "food", 
               "travel", "movie", "sports", "game", "food"],
    parent=["", "positive", "positive", "before", "before", "before","before","before", 
            "after","after","after","after","after"],
    value=[0, 1.78, 1.813, 0.395, 0.289, 0.343, 0.359, 0.395, 0.362,0.307,0.356,0.399,0.388])

fig = px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
)
fig.show()

In [5]:
import pyspark as spark
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.sql.functions import col,udf,monotonically_increasing_id,unix_timestamp,round,avg
import re
sc = spark.SparkContext()
sql = spark.SQLContext(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


KeyboardInterrupt: 

In [2]:
travel_before = pd.read_csv('data/travel_before_covid.csv')
travel_after = pd.read_csv('data/travel_after_covid.csv')
sports_before = pd.read_csv('data/sports_before_covid.csv')
sports_after = pd.read_csv('data/sports_after_covid.csv')
movie_before = pd.read_csv('data/movie_before_covid.csv')
movie_after = pd.read_csv('data/movie_after_covid.csv')
game_before = pd.read_csv('data/game_before_covid.csv')
game_after = pd.read_csv('data/game_after_covid.csv')
food_before = pd.read_csv('data/food_before_covid.csv')
food_after = pd.read_csv('data/food_after_covid.csv')

In [7]:
travel_before = pd.DataFrame(travel_before, columns=['Date', 'Content'])
travel_after = pd.DataFrame(travel_after, columns=['Date', 'Content'])
sports_before = pd.DataFrame(sports_before, columns=['Date', 'Content'])
sports_after = pd.DataFrame(sports_after, columns=['Date', 'Content'])
movie_before = pd.DataFrame(movie_before, columns=['Date', 'Content'])
movie_after = pd.DataFrame(movie_after, columns=['Date', 'Content'])
game_before = pd.DataFrame(game_before, columns=['Date', 'Content'])
game_after = pd.DataFrame(game_after, columns=['Date', 'Content'])
food_before = pd.DataFrame(food_before, columns=['Date', 'Content'])
food_after = pd.DataFrame(food_after, columns=['Date', 'Content'])

In [15]:
data=food_after
cdata=pd.DataFrame(columns=['Date','Content'])
index=0
for index,row in data.iterrows():
    stre=row["Content"]
    #the following cleaning steps are not the standard. You may generate your own ways of cleaning.
    my_new_string = re.sub('#\w+', '', stre) #remove the hashtags
    my_new_string = re.sub('http\S+', '', my_new_string) #remove the links
    my_new_string = re.sub('\n', '', my_new_string) #remove \n
    cdata.sort_index()
    cdata.at[index,'Date']=row["Date"]
    cdata.at[index,'Content']=my_new_string
    index=index+1
food_after=cdata
food_after['Date'] = food_after['Date'].astype('datetime64[ns]')
food_after["Date"]=food_after["Date"].dt.date

In [ ]:
travel_before.head()

In [52]:
FullDataTw=sql.createDataFrame(game_after)
FullDataTw = FullDataTw.dropna()
FullDataTw.select(monotonically_increasing_id().alias("rowId"),"*")
 #setting column names of Twitter dataset
CleanDF = FullDataTw.withColumnRenamed('Contents', 'Content')
CleanDF = FullDataTw.withColumnRenamed('Date', 'Date_Time')
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()
def senti_score_udf(sentence):
    snt = analyser.polarity_scores(sentence)
    return ([snt['neg'], snt['neu'], snt['pos'], snt['compound']])
func_udf2 = udf(senti_score_udf, ArrayType(FloatType()))
CleanDF = CleanDF.withColumn('p_neg', func_udf2(CleanDF['Content'])[0])  #withColumn function would generate the result with a new column name
CleanDF = CleanDF.withColumn('p_neu', func_udf2(CleanDF['Content'])[1])
CleanDF = CleanDF.withColumn('p_pos', func_udf2(CleanDF['Content'])[2])
CleanDF = CleanDF.withColumn('p_comp', func_udf2(CleanDF['Content'])[3])
CleanDF.show(120)
CleanDF.toPandas().to_csv('game_after_sen.csv',index=False)

+----------+-------------------------------------+-----+-----+-----+-------+
| Date_Time|                              Content|p_neg|p_neu|p_pos| p_comp|
+----------+-------------------------------------+-----+-----+-----+-------+
|2020-05-31|                 WIN a Brand New A...|  0.0|0.788|0.212| 0.7284|
|2020-05-31|                 What an awesome t...|  0.0| 0.71| 0.29| 0.8469|
|2020-05-31|                 WHERE did this gu...|  0.0|  1.0|  0.0|    0.0|
|2020-05-31|                 5 Untamed Packs G...|0.038|0.665|0.297| 0.8472|
|2020-05-31|                 Play Word Collect...|  0.0|0.748|0.252| 0.4003|
|2020-05-31|                 Bagus! Aku memeli...|  0.0|  1.0|  0.0|    0.0|
|2020-05-31|                 31-05-20 Sunday n...|  0.0|  1.0|  0.0|    0.0|
|2020-05-31|                 What an awesome t...|  0.0| 0.71| 0.29| 0.8469|
|2020-05-31|                 Y en otras notici...|0.064|0.603|0.333| 0.8415|
|2020-05-31|                 SWhat an awesome ...|  0.0| 0.71| 0.29| 0.8469|

In [49]:
tw=pd.read_csv("game_before_sen.csv",lineterminator="\n")
tw["length"]=""
i=0
# iterate through the csv file 
for val in tw["Content"]: 
  val = str(val) 
  tokens = val.split() 
  tw["length"][i]=len(tokens)
  i=i+1
tw=tw.drop(columns=["p_neg","p_neu","p_pos"])

In [50]:
tw.loc[(tw['p_comp']>-0.05)&(tw['p_comp']<0.05),'sentiment']="neutral"
tw.loc[(tw['p_comp']>=0.05)&(tw['p_comp']<0.9),'sentiment']='positive'
tw.loc[(tw['p_comp']<=-0.05)&(tw['p_comp']>-0.9),'sentiment']='negative'
tw.loc[tw['p_comp']<=-0.9,'sentiment']='extremely negative'
tw.loc[tw['p_comp']>=0.9,'sentiment']='extremely positive'
tw.to_csv('game_before_p_comp.csv')